In [1]:
import pandas as pd
import numpy as np

In [2]:
def test():
    data = pd.read_csv("human.csv")
    data['gender'] = data['gender'].map({'M':0,'F':1})
    data.rename(columns = {'Unnamed: 0':'Name'}, inplace=True)
    
    return data

def dataset():
    data = pd.read_csv("simpsons.csv")
    data['gender'] = data['gender'].map({'M':0,'F':1})
    data.rename(columns = {'Unnamed: 0':'Name'}, inplace=True)
    data = data.set_index(data.columns[0])
    features = data.columns
    
    return data,features

In [3]:
import math
def get_entropy(data,features):
    global entropy
    
    answer = data.groupby(features[-1]).count()
    
    if answer.empty :
        male_entropy = 0
        female_entropy = 0
    else:
        if answer.index[0] == 0 and len(answer.index) == 2:
            male = answer.values[0][0]
            female = answer.values[1][0]
            total = male + female
            male_entropy = (male / total) * (-1 * (math.log2(male/total)))
            female_entropy = (female / total) * (-1 * (math.log2(female/total)))
        else:        
            if answer.index[0] == 0:
                male = answer.values[0][0]
                total = male + 0
                male_entropy = (male / total) * (-1 * (math.log2(male/total)))
                female_entropy = 0
                
            elif answer.index[0] == 1:
                female = answer.values[0][0]
                total = 0 + female
                male_entropy = 0
                female_entropy = (female / total) * (-1 * (math.log2(female/total)))

        entropy = male_entropy + female_entropy

    return entropy

In [4]:
def get_condition_entropy(data_values,feature):
    x = data[data[feature] >= data_values]
    y = data[data[feature] < data_values]
    
    x_entropy = get_entropy(x)
    y_entropy = get_entropy(y)
    
    xy_entropy = (x_entropy * len(x)/len(x+y)) + (y_entropy * len(y)/len(x+y))


    w = data[data[feature] > data_values]
    z = data[data[feature] <= data_values]
    
    w_entropy = get_entropy(w)
    z_entropy = get_entropy(z)
    
    wz_entropy = (w_entropy * len(w)/len(w+z)) + (z_entropy * len(z)/len(w+z))
    
    return xy_entropy, wz_entropy

In [5]:
def get_best_entropy(fs_entropy,sd_entropy,i,j):
    fs_con = np.ones(features.size -1)
    sd_con = np.ones(features.size -1)
    
    fs_index = np.zeros(features.size -1)
    sd_index = np.zeros(features.size -1)
    
    if fs_con[i] > fs_entropy:
        fs_con[i] = fs_entropy
        fs_index[i] = j
        
    if sd_con[i] > sd_entropy:
        sd_con[i] = sd_entropy
        sd_index[i] = j
        
    return fs_con,fs_index,sd_con,sd_index

In [6]:
#최적의 몸무게를 찾는 러닝
def get_info():
    best_entropy = np.ones(features.size -1)
    best_index = np.zeros(features.size -1, dtype = 'i')
    operators = np.zeros(features.size -1, dtype = 'U4')
    
    for i in range(0,features.size-1):
        for j in range(0,len(data)):
            fs_entropy, sd_entropy = get_condition_entropy(data[features[i]][j],features[i])
            fs_con,fs_index,sd_con,sd_index = get_best_entropy(fs_entropy,sd_entropy,i,j)
            
    for k in range(0,features.size-1):
        if fs_con[k] >= sd_con[k]:
            best_entropy[k] = sd_con[k]
            best_index[k]= sd_index[k]
            operators[k] = ">"
        else:
            best_entropy[k] = fs_con[k]
            best_index[k] = fs_index[k]
            operators[k] = "<"
            
    return best_entropy,best_index,operators

In [7]:
#기준으로 나뉜 데이터 셋 두개 지정
import operator as op

ops = {"<" : op.lt,
       ">" : op.gt
      }

left_data = []
right_data = []

def split_dataset(condition):
    global left_dataset, right_dataset
    
    for i in range(0,len(data)):
        if condition[i] == True :
            left_data.append(i)
        else:
            right_data.append(i)

    left_dataset = data.iloc[left_data].reset_index(drop = True)
    
    right_dataset = data.iloc[right_data].reset_index(drop = True)
    
    return left_dataset, right_dataset

In [8]:
# 조건을 찾고 그의 맞는 데이터 셋을 2개로 분할
def find_conditions():
    
    left_data.clear()
    right_data.clear()
    
    best_entropy,best_index,operators = get_info()
    best_condition = best_entropy.argmin()
    
    condition = ops[operators[best_condition]](data[features[best_condition]],
                data[features[best_condition]][best_index[best_condition]])
    
    left_dataset,right_dataset = split_dataset(condition)
    confirm_feature = features[best_codition]
    
    return left_dataset,right_dataset


In [9]:
#decision tree learning
right_datasets = []
i = 0
def learning():
    global data,i
#     data = test()
    data, features = dataset()
    print("Original Data:\n",data,"\n")
    right_datasets.clear()
    left_dataset, right_dataset = find_conditions()
    print("left_node: \n",left_dataset,"\n")
    print("right_node: \n",right_dataset,"\n")
    while True:
        if find_entropy(left_dataset) != 0:
            right_datasets.insert(0, right_dataset)
            i += 1
            data = left_dataset
            left_dataset, right_dataset = find_conditions()
            print("left_node: \n",left_dataset,"\n")
            print("right_node: \n",right_dataset,"\n")
        else:
            if find_entropy(right_dataset) != 0:
                data = right_dataset
                left_dataset, right_dataset = find_conditions()
                print("left_node: \n",left_dataset,"\n")
                print("right_node: \n",right_dataset,"\n")
            else:
                if not right_datasets:
                    break
                data = right_datasets.pop()
                i += -1
                left_dataset, right_dataset = find_conditions()
                print("left_node: \n",left_dataset,"\n")
                print("right_node: \n",right_dataset,"\n")
                    
                if find_entropy(left_dataset) == 0 and find_entropy(right_dataset) == 0:
                    if find_entropy(right_datasets[-1]) == 0:
                        data = right_datasets.pop()
                        i += -1
                        if not right_datasets:
                            print(data)
                            break;
                        else:
                            left_dataset, right_dataset = find_conditions()

In [11]:
def decision_tree():
    learning()
decision_tree()

Original Data:
         hair_length  weight  age  gender
Name                                    
Homer             0     250   36       0
Marge            10     150   34       1
Bart              2      90   10       0
Lisa              6      78    8       1
Maggie            4      20    1       1
Abe               1     170   70       0
Selma             8     160   41       1
Otto             10     180   38       0
Krusty            6     200   45       0 



NameError: name 'features' is not defined

In [92]:
b = data.columns[1:]

In [93]:
data[data[b[0]] >= 5]


,hair_length,weight,age,gender
Name,,,,
Homer,0,250,36,0
Marge,10,150,34,1
Bart,2,90,10,0
Lisa,6,78,8,1
Maggie,4,20,1,1
Abe,1,170,70,0
Selma,8,160,41,1
Otto,10,180,38,0
Krusty,6,200,45,0


In [94]:

data.groupby(b[-1]).count()

,hair_length,weight,age
gender,,,
0,5,5,5
1,4,4,4


In [112]:
s,e= dataset()

e

Index(['hair_length', 'weight', 'age', 'gender'], dtype='object')